# Métodos Estatísticos em Data Mining
### Projeto 1

##### Grupo 10:
- 88174 Marisa Pereira 
- 89616 Beatriz Silva  
- 89629 João Batista 
- 89640 Tomás Freire


April 25st, 2021

# 1. Libraries

In [ ]:
#install.packages("mice")
#install.packages("corrplot")
#install.packages("ggplot2")
#install.packages("psych")
#install.packages("MASS")
#install.packages("comprehenr")
#install.packages("rpart")
#install.packages("rpart.plot")
#install.packages("caTools")
#install.packages("neuralnet")
#install.packages("logisticPCA")
#install.packages("mlr") 
#install.packages("imbalance") 
#install.packages("caret")
#install.packages("sats")
#install.packages("dplyr")
#install.packages("tidyverse")
#install.packages("funModeling")
#install.packages("Hmisc")
#install.packages("factoextra")
#install.packages("ggplot2")
#install.packages("grid")
#install.packages("devtools")
#install.packages("factoextra")
#install.packages("e1071")
#install.packages("caret")
#install.packages("pROC")
#install.packages("base")
#install.packages("MLmetrics")
#install.packages("nnet")
#install.packages("rpart")
#install.packages("rpart.plot")
#install.packages("klaR")
#install.packages("imbalance")

In [ ]:
library("stats")
library("dplyr")
library("tidyverse")
library("funModeling")
library("Hmisc")
library("ggplot2")
library("grid")
library("factoextra")
library("devtools")
library("plyr")
library("factoextra")
library("e1071")
library("caret")
library("pROC")
library("base")
library("MLmetrics")
library("nnet")
library("rpart")
library("rpart.plot")
library("klaR")
library("imbalance")
library("MASS")
library("comprehenr")

# 2. Load the Dataset

In [ ]:
df <- read.csv("Myocardial-infarction-complications-Database.csv")
#tirar as variaveis que nao usamos no nosso problema
df <- df[c(1:93,96:100,103,106:112,119)]

#Remove all dimensions with more that 35% of missing values: (NA_KB; NOT_NA_KB; LID_KB; IBS_NASL;S_AD_KBRIG; D_AD_KBRIG; KFK_BLOOD)
data <- df[-c(8,94,95,96,89,35,36)]


# 3. Exploratory Data Analysis

In [ ]:
#summary
summary(df)
#descriptive statistics
round(describe(df),1)

## 3.1 Handling Missing Values

In [ ]:
#dealing with missing values:
library("mice")

#Applying Predictive Mean Matching 5 times and choosing one of them
imp <- mice(data[,-c(1)], m = 5, method = "pmm",seed=20); # Impute missing values
data_imp <- complete(imp,2);
data_imp <- cbind(data[,c(1)],data_imp);

## 3.2 Correlation
#### Conclusion: 
- Variables S_AD_ORIT and D_AD_ORIT have a correlation of 0.86, so here we Remove the first one;

In [ ]:
#Variables S_AD_ORIT and D_AD_ORIT have a correlation of 0.86, so here we Remove the first one.
data_sem_corr <- data_imp[,-c(34)];

## 3.3 Creating Dummy Variables

In [ ]:
#tirar as variaveis continuas
data_sem_continuas <-  data_sem_corr[,-c(1,2,34,80,82,83,84,85,86,99)]

#passar as variaveis para "factor":
for (i in 1:length(data_sem_continuas)) {
    #data$Region <- as.factor(data$Region)
    data_sem_continuas[,i] <- as.factor(data_sem_continuas[,i])
    
    }

#criar dummy variables:
library("mlr")

data_com_dummies <- createDummyFeatures(data_sem_continuas, method = "reference")

## 3.4 PCA

In [ ]:
#fazer logistic PCA:
library("logisticPCA")


dimension = 25
#logpca_cv.25<-cv.lpca(data_com_dummies, ks = dimension, ms = 1:13)
#plot(logpca_cv.25)
logpca_model.25<-logisticPCA(data_com_dummies, k = dimension, m = 11)
#logpca_model.25


#o "logpca_model.25$PCs", que tem a informação, é estranho... vamos transformar isto numa matriz de jeito
data_25 <- matrix(0, nrow = 1700, ncol = 25)
for (i in 1:25) {
    #data$Region <- as.factor(data$Region)
    data_25[,i] <- logpca_model.25$PCs[,i]
    
    }

#repor as variaveis continuas e a de resposta:

data_final <- cbind(data_sem_corr[,c(1,2)], data_25)
data_final <- cbind(data_final, data_sem_corr[,c(34,80,82,83,84,85,86,99)])


#se as variaveis nao tivessem os mesmos nomes dava erro so...
colnames(data_final)[2] <- "AGE"
colnames(data_final)[3] <- "X1"
colnames(data_final)[4] <- "X2"
colnames(data_final)[5] <- "X3"
colnames(data_final)[6] <- "X4"
colnames(data_final)[7] <- "X5"
colnames(data_final)[8] <- "X6"
colnames(data_final)[9] <- "X7"
colnames(data_final)[10] <- "X8"
colnames(data_final)[11] <- "X9"
colnames(data_final)[12] <- "X10"
colnames(data_final)[13] <- "X11"
colnames(data_final)[14] <- "X12"
colnames(data_final)[15] <- "X13"
colnames(data_final)[16] <- "X14"
colnames(data_final)[17] <- "X15"
colnames(data_final)[18] <- "X16"
colnames(data_final)[19] <- "X17"
colnames(data_final)[20] <- "X18"
colnames(data_final)[21] <- "X19"
colnames(data_final)[22] <- "X20"
colnames(data_final)[23] <- "X21"
colnames(data_final)[24] <- "X22"
colnames(data_final)[25] <- "X23"
colnames(data_final)[26] <- "X24"
colnames(data_final)[27] <- "X25"

#é preciso mudar o nome da variavel resposta par "Class" para o oversample funcionar
data_final$Class <- data_sem_corr[,c(99)]
data_final <- data_final[,-c(35)]

## 3.4 Oversampling

In [ ]:
#fazer oversample:
#o site diz: MWMOTE is an extension of the original SMOTE algorithm. It assigns higher weight to borderline instances, 
#undersized minority clusters and examples near the borderline of the two classes.

newMWMOTE <- mwmote(data_final[,-c(1)], numInstances = 300)
data_oversample <- rbind(data_final[,-c(1)],newMWMOTE)


In [ ]:
data_final[,35]

### Proof that dimension 25 is better and with m=11

In [ ]:
#dimension = 25
#logpca_cv.26<-cv.lpca(data_com_dummies, ks = dimension, ms = 1:13)
#plot(logpca_cv.26)
#logpca_model.26<-logisticPCA(data_com_dummies, k = dimension, m = which.min(logpca_cv.26))
#logpca_model.26

In [ ]:
#dimension = 24
#logpca_cv.26<-cv.lpca(data_com_dummies, ks = dimension, ms = 1:13)
#plot(logpca_cv.26)
#logpca_model.26<-logisticPCA(data_com_dummies, k = dimension, m = which.min(logpca_cv.26))

#logpca_model.26

# 4. Methods

## 4.1 Final Data Set

In [ ]:
data_final<-data_final[,-c(1)]

In [ ]:
colnames(data_final)[34] <- "resposta"


data_final$resposta<- as.factor(data_final$resposta)

set.seed(5968)
indxTrain <- createDataPartition(y = data_final$resposta ,p = 0.9,list = FALSE)
data.train <- data_final[indxTrain,]
data.test <- data_final[-indxTrain,] 
# Setting levels for both training and test data
levels(data.train$resposta) <- make.names(levels(factor(data.train$resposta)))
levels(data.test$resposta) <- make.names(levels(factor(data.test$resposta)))
data.test$resposta <- factor(data.test$resposta , levels=rev(levels(data.test$resposta)))
data.train$resposta <- factor(data.train$resposta , levels=rev(levels(data.train$resposta)))

In [ ]:
x = caret::trainControl(method = 'cv',
                        number = 10,
                        classProbs = TRUE,
                        summaryFunction = prSummary
)

In [ ]:
x_nn = caret::trainControl(method = 'cv',
                        number = 10,
                        classProbs = TRUE
)

## Naive Bayes Classifier

In [ ]:
modelnb<-caret::train(resposta~. , data = data.train, method = 'nb',trControl = x,  
                tuneLength = 10, metric = "F")
modelnb
plot(modelnb)

#confusion matrix and statistics for the model 
test_pred_nb <- predict(modelnb, newdata = data.test)
confusionMatrix(test_pred_nb, data.test$resposta, mode="everything", positive="X1")

roc.multi_nb<-multiclass.roc(data.test$resposta,as.numeric(test_pred_nb))
roc.multi_nb$auc
#rs <- roc.multi[['rocs']]
#plot.roc(rs[[1]])

## KNN

In [ ]:
#k-NN classifier
n_vizinhos = c(1,3,5,7,9,11,13,15,17,19,21,23)
modelknn<- caret::train(resposta~. , data = data.train, method = 'knn',
                          tuneGrid   = expand.grid(k = n_vizinhos),
                          trControl = x,  
                          tuneLength = 10,
                       metric = "F")
# Summary of model 1(no standardised data)
modelknn
plot(modelknn)

#confusion matrix and statistics for the model 
test_pred_knn <- predict(modelknn, newdata = data.test)
confusionMatrix(test_pred_knn, data.test$resposta, mode="everything", positive="X1")
roc.multi_knn<-multiclass.roc(data.test$resposta,as.numeric(test_pred_knn))
roc.multi_knn$auc
#rs <- roc.multi[['rocs']]
#plot.roc(rs[[1]])

## Multinomial Logistic Regression

In [ ]:
#logistic regression
modelmlr <- caret::train(resposta~. , data = data.train, method = 'multinom',
                 trControl = x,  
                 tuneLength = 10,
                metric = "F")
#modelmlr
plot(modelmlr)

#confusion matrix and statistics for the model 
test_pred_blr <- predict(modelmlr, newdata = data.test)
confusionMatrix(test_pred_blr, data.test$resposta,mode="everything", positive="X1")
roc.multi_blr<-multiclass.roc(data.test$resposta,as.numeric(test_pred_blr))
roc.multi_blr$auc
#rs <- roc.multi[['rocs']]
#plot.roc(rs[[1]])

## Neural Networks

In [ ]:
modelnn <- caret::train(resposta~. , data = data.train, method = 'nnet',trControl = x_nn,  
                tuneLength = 10)
#modelnn
plot(modelnn)
test_pred_nn <- predict(modelnn, newdata = data.test)
confusionMatrix(test_pred_nn, data.test$resposta, mode="everything", positive="X1")
roc.multi_nn<-multiclass.roc(data.test$resposta,as.numeric(test_pred_nn))
roc.multi_nn$auc

## LDA

In [ ]:
modellda <- caret::train(resposta~.,
                      method     = "lda",
                      trControl  = x_nn,
                      data       = data.train)

test_pred_lda <- predict(modellda, newdata = data.test)
confusionMatrix(test_pred_lda, data.test$resposta,mode="everything", positive="X1")
roc.multi_lda<-multiclass.roc(data.test$resposta,as.numeric(test_pred_lda))
roc.multi_lda$auc

## LDA Cross

In [ ]:
#função que treina o linear discriminant analises:
library("MASS")
library("comprehenr")

LDA <- function(data,response){
    
data_aux <- data

#function that trains the linear discriminator, with leave one out cross validation
fit  <- lda(x = data_aux[,-c(response)],grouping = data_aux[,c(response)],CV=TRUE)

#calculating the criteria, accuracy, recalls, precisions and F1-score
print("confusion matrix: first collum -> are truly negative;  second line -> are classified as positive")
tt<-table(fit$class,data_aux[,c(response)]);print(tt)
print("accuracy:")
acc<-sum(diag(tt))/sum(tt);print(acc)
print("recalls:")
recalls <- to_vec(for(i in 1:2) if(TRUE) diag(tt)[i]/sum(tt[,i]));print(recalls)
print("precisions:")
precisions  <- to_vec(for(i in 1:2) if(TRUE) diag(tt)[i]/sum(tt[i,]));print(precisions)
print("F1_measures:")
F1_measure  <-to_vec(for(i in 1:2) if(TRUE) 2*recalls[i]*precisions[i]/(recalls[i]+precisions[i])  );print(F1_measure)
print("Overall F1_measure:")
F1 <- mean(c(F1_measure));print(F1)
    
}

In [ ]:
#subtrair a média a todos os valores
data_aux <- data_final[,-c(34)]

nomes <- c()

data_final_centered <- matrix(0, nrow = length(data_aux[,c(1)]), ncol = length(data_aux))
for (i in 1:length(data_aux)) {
    mean_aux = mean(data_aux[,c(i)])
    for (j in 1:length(data_aux[,1])){
        
        data_final_centered[j,i] = data_aux[j,i] - mean_aux 
        }
    }
#fazer os cross products

data_aux <- data_final_centered
data_final_cross <- matrix(0, nrow = length(data_aux[,1]), ncol = length(data_aux[1,]) * (length(data_aux[1,])+1) / 2)
for (i in 1:length(data_aux[1,])) {
    
    for (j in i:length(data_aux[1,])){
        tamanho_aux = length(data_aux[1,]) + 1
        
        m <-  (tamanho_aux-i+1):tamanho_aux; #sum(m) 
        
        n = sum(m) - tamanho_aux + j - i + 1
        
        data_final_cross[,n] = data_aux[,i] * data_aux[,j]
        
        nomes <- c(nomes, paste(colnames( data_final[,-c(34)])[i], colnames( data_final[,-c(34)])[j]))
    }
}
colnames(data_final_cross) <- nomes
data_final_cross <- cbind(data_final_cross, data_final[,c(34)])
colnames(data_final_cross)[562] <- "resposta_cross"

In [ ]:
LDA(data_final_cross,562)

## QDA

In [ ]:
modelqda <- caret::train(resposta~.,
                      method     = "qda",
                      trControl  = x_nn,
                      data       = data.train)

test_pred_qda <- predict(modelqda, newdata = data.test)
confusionMatrix(test_pred_qda, data.test$resposta,mode="everything", positive="X1")
roc.multi_qda<-multiclass.roc(data.test$resposta,as.numeric(test_pred_qda))
roc.multi_qda$auc

## SVM

In [ ]:
modelsvm<- caret::train(resposta~. , data = data.train, method = 'svmRadial',
                 trControl = x, preProcess = c("center","scale"),  
                 tuneLength = 10)
#modelsvm
plot(modelsvm)

#confusion matrix and statistics for the model 
test_pred_svm <- predict(modelsvm, newdata = data.test)
confusionMatrix(test_pred_svm, data.test$resposta,mode="everything", positive="X1")
roc.multi_svm<-multiclass.roc(data.test$resposta,as.numeric(test_pred_svm))
roc.multi_svm$auc
#rs <- roc.multi[['rocs']]
#plot.roc(rs[[1]])

## 4.2 Oversample Data set

In [ ]:
colnames(data_oversample)[34] <- "resposta"

set.seed(2021)
data_oversample$resposta<- as.factor(data_oversample$resposta)

indxTrain <- createDataPartition(y = data_oversample$resposta ,p = 0.90,list = FALSE)
data.train_over <- data_oversample[indxTrain,]
data.test_over <- data_oversample[-indxTrain,] 

# Setting levels for both training and test data
levels(data.train_over$resposta) <- make.names(levels(factor(data.train_over$resposta)))
levels(data.test_over$resposta) <- make.names(levels(factor(data.test_over$resposta)))
data.test_over$resposta <- factor(data.test_over$resposta , levels=rev(levels(data.test_over$resposta)))
data.train_over$resposta <- factor(data.train_over$resposta , levels=rev(levels(data.train_over$resposta)))

In [ ]:
x = caret::trainControl(method = 'cv',
                        number = 10,
                        classProbs = TRUE,
                        summaryFunction = prSummary
)

In [ ]:
x_nn = caret::trainControl(method = 'cv',
                        number = 10,
                        classProbs = TRUE
)

## Naive Bayes

In [ ]:
modelnb_over <- caret::train(resposta~. , data = data.train_over, method = 'nb',trControl = x,  
                tuneLength = 10, metric="F")
modelnb_over
plot(modelnb_over)

#confusion matrix and statistics for the model 
test_pred_nb_over <- predict(modelnb_over, newdata = data.test_over)
confusionMatrix(test_pred_nb_over, data.test_over$resposta, mode="everything", positive="X1")
roc.multi_nb_over<-multiclass.roc(data.test_over$resposta,as.numeric(test_pred_nb_over))
roc.multi_nb_over$auc
#rs <- roc.multi[['rocs']]
#plot.roc(rs[[1]])

## KNN

In [ ]:
#k-NN classifier
n_vizinhos = c(1,3,5,7,9,11,13,15,17,19,21,23)
modelknn_over<- caret::train(resposta~. , data = data.train_over, method = 'knn',
                          tuneGrid   = expand.grid(k = n_vizinhos),
                          trControl = x,  
                          tuneLength = 10,
                        metric="F")
# Summary of model 1(no standardised data)
modelknn_over
plot(modelknn_over)

#confusion matrix and statistics for the model 
test_pred_knn_over <- predict(modelknn_over, newdata = data.test_over)
confusionMatrix(test_pred_knn_over, data.test_over$resposta, mode="everything", positive="X1")
roc.multi_knn_over<-multiclass.roc(data.test_over$resposta,as.numeric(test_pred_knn_over))
roc.multi_knn_over$auc
#rs <- roc.multi[['rocs']]
#plot.roc(rs[[1]])

## Multinomial Logistic Regression

In [ ]:
modelmlr<- caret::train(resposta~. , data = data.train_over, method = 'multinom',
                 trControl = x,  
                 tuneLength = 10,
                       metric = "F")
#modelmlr
plot(modelmlr)

#confusion matrix and statistics for the model 
test_pred_blr_over <- predict(modelmlr, newdata = data.test_over)
confusionMatrix(test_pred_blr_over, data.test_over$resposta,mode="everything", positive="X1")
roc.multi_blr_over<-multiclass.roc(data.test_over$resposta,as.numeric(test_pred_blr_over))
roc.multi_blr_over$auc
#rs <- roc.multi[['rocs']]
#plot.roc(rs[[1]])

## Neural Networks

In [ ]:
modelnn_over <- caret::train(resposta~. , data = data.train_over, method = 'nnet',trControl = x_nn,  
                tuneLength = 10)
#modelnn_over
plot(modelnn_over)
test_pred_nn_over <- predict(modelnn_over, newdata = data.test_over)
confusionMatrix(test_pred_nn_over, data.test_over$resposta, mode="everything", positive="X1")
roc.multi_nn_over<-multiclass.roc(data.test_over$resposta,as.numeric(test_pred_nn_over))
roc.multi_nn_over$auc

## LDA

In [ ]:
modellda_over <- caret::train(resposta~.,
                      method     = "lda",
                      trControl  = x_nn,
                      data       = data.train_over)

test_pred_lda_over <- predict(modellda_over, newdata = data.test_over)
confusionMatrix(test_pred_lda_over, data.test_over$resposta,mode="everything", positive="X1")
roc.multi_lda_over<-multiclass.roc(data.test_over$resposta,as.numeric(test_pred_lda_over))
roc.multi_lda_over$auc
#rs <- roc.multi_lda_over[['rocs']]
#plot.roc(rs[[1]])

## LDA Cross

In [ ]:
#subtrair a média a todos os valores
data_aux <- data_oversample[,-c(34)]

data_oversample_centered <- matrix(0, nrow = length(data_aux[,c(1)]), ncol = length(data_aux))

nomes <- c()

for (i in 1:length(data_aux)) { 
    mean_aux = mean(data_aux[,c(i)])
    
    for (j in 1:length(data_aux[,1])){  
        data_oversample_centered[j,i] = data_aux[j,i] - mean_aux 
        }
    }
#fazer os cross products

data_aux <- data_oversample_centered
data_oversample_cross <- matrix(0, nrow = length(data_aux[,1]), ncol = length(data_aux[1,]) * (length(data_aux[1,])+1) / 2)
for (i in 1:length(data_aux[1,])) {
    for (j in i:length(data_aux[1,])){
        tamanho_aux = length(data_aux[1,]) + 1
        
        m <-  (tamanho_aux-i+1):tamanho_aux; #sum(m) 
        
        n = sum(m) - tamanho_aux + j - i + 1
        
        data_oversample_cross[,n] = data_aux[,i] * data_aux[,j]

        nomes <- c(nomes, paste(colnames( data_oversample[,-c(34)])[i], colnames( data_oversample[,-c(34)])[j]))

    }
}
colnames(data_oversample_cross) <- nomes
data_oversample_cross <- cbind(data_oversample_cross, data_oversample[,c(34)])
colnames(data_oversample_cross)[562] <- "resposta_cross"

In [ ]:
LDA(data_oversample_cross,562)

## QDA

In [ ]:
modelqda_over <- caret::train(resposta~.,
                      method     = "qda",
                      trControl  = x_nn,
                      data       = data.train_over)

test_pred_qda_over <- predict(modelqda_over, newdata = data.test_over)
confusionMatrix(test_pred_qda_over, data.test_over$resposta,mode="everything", positive="X1")
roc.multi_qda_over<-multiclass.roc(data.test_over$resposta,as.numeric(test_pred_qda_over))
roc.multi_qda_over$auc
#rs <- roc.multi_qda_over[['rocs']]
#plot.roc(rs[[1]])

## SVM

In [ ]:
modelsvm_over <- caret::train(resposta~. , data = data.train_over, method = 'svmRadial',
                 trControl = x, preProcess = c("center","scale"),  
                 tuneLength = 10,
                       metric = "F")
#modelsvm
plot(modelsvm_over)

#confusion matrix and statistics for the model 
test_pred_svm_over <- predict(modelsvm_over, newdata = data.test_over)
confusionMatrix(test_pred_svm_over, data.test_over$resposta,mode="everything", positive="X1")
roc.multi_svm_over<-multiclass.roc(data.test_over$resposta,as.numeric(test_pred_svm_over))
roc.multi_svm_over$auc
#rs <- roc.multi[['rocs']]
#plot.roc(rs[[1]])

## 4.3 Decision Trees

In [ ]:
library("rpart")
library("rpart.plot")
library("caTools")
tree <- function(data,response){
    
data_aux <- data

#we will add the measure of every simulation and then average in the end
#this code initializes the variables that will keep the added measures.
tt_tree_aux <- table(c(0,1), c(0,1)) - table(c(0,1), c(0,1))
acc_tree_aux <- 0
recalls_tree_aux <- c(0,0)
precisions_tree_aux  <- c(0,0)
F1_measure_tree_aux  <- c(0,0)
F1_tree_aux <- 0

#this cycle will train a tree 20 times and then average the criteria
number_of_cycles <- 20
for (i in 1:number_of_cycles) {
  
  #split the data into training set and testing set
  spl = sample.split(data_aux, SplitRatio = 0.90)
  train = subset(data_aux, spl==TRUE)
  test = subset(data_aux, spl==FALSE)
  
  #training the tree using the training set
  fit_tree <- rpart(train[,c(response)]~., data = train[,-c(response)], method = 'class');fit_tree

  #predictions on the testing set
  predictions_tree <- predict(fit_tree, newdata = test[,-c(response)], type = 'class')
  
  #calculating the criteria, accuracy, recalls, precisions and F1-score
  tt_tree <- table(predictions_tree, test[,c(response)])
  acc_tree <- sum(diag(tt_tree))/sum(tt_tree)
  recalls_tree <- to_vec(for(i in 1:2) if(TRUE) diag(tt_tree)[i]/sum(tt_tree[,i]))
  precisions_tree  <- to_vec(for(i in 1:2) if(TRUE) diag(tt_tree)[i]/sum(tt_tree[i,]))
  F1_measure_tree  <- to_vec(for(i in 1:2) if(TRUE) 2*recalls_tree[i]*precisions_tree[i]/(recalls_tree[i]+precisions_tree[i])  )
  F1_tree <- mean(c(F1_measure_tree))
  
  
  #This is to add the measure with previous simulations, in the end the average will be made
  tt_tree_aux <- tt_tree_aux + tt_tree
  acc_tree_aux <- acc_tree_aux + acc_tree
  recalls_tree_aux <- recalls_tree_aux + recalls_tree
  precisions_tree_aux  <- precisions_tree_aux + precisions_tree
  F1_measure_tree_aux  <- F1_measure_tree_aux + F1_measure_tree
  F1_tree_aux <- F1_tree_aux + F1_tree
}
#this plots the last calculated tree
rpart.plot(fit_tree,box.palette = colorRampPalette(c("#F55118","#FF9933","#FFFFFF","#66CCCC","#298DF7"))(50))

#finaly, the criteria are averaged out
print("confusion matrix: first collum -> are truly positive;  second line -> are classified as negative")
tt_tree_final <- tt_tree_aux/number_of_cycles;print(tt_tree_final)
print("accuracy:")
acc_tree_final <- acc_tree_aux/number_of_cycles;print(acc_tree_final)
print("recalls:")  
recalls_tree_final <- recalls_tree_aux/number_of_cycles;print(recalls_tree_final)
print("precisions:")
precisions_tree_final  <- precisions_tree_aux/number_of_cycles;print(precisions_tree_final)
print("F1_measures:")
F1_measure_tree_final  <- F1_measure_tree_aux/number_of_cycles;print(F1_measure_tree_final)
print("Overall F1_measure:")
F1_tree_final <- F1_tree_aux/number_of_cycles;
print(F1_tree_final)
}

In [ ]:
tree(data_final,34)

In [ ]:
tree(data_oversample,34)